In [ ]:
import os

# Set the input video path to an environment variable
os.environ['TARGET_VIDEO_PATH']='data/<<>>.h264'
os.environ['TARGET_VIDEO_PATH_MP4']='<<>>.mp4'

target_video_path=os.environ['TARGET_VIDEO_PATH']
target_video_path_mp4=os.environ['TARGET_VIDEO_PATH_MP4']

# Analyze video
!ffprobe -i $TARGET_VIDEO_PATH \
         -hide_banner
!ffprobe -i $TARGET_VIDEO_PATH_MP4 \
         -hide_banner

In [ ]:
from IPython.display import Video

# Convert the H.264 encoded video file to MP4 container file - this will generate the sample_30.mp4 file
!ffmpeg -i $TARGET_VIDEO_PATH $TARGET_VIDEO_PATH_MP4 -y -loglevel quiet

# View the input video
Video(target_video_path_mp4, width=720)

In [ ]:
import os
os.environ['NGC_DIR']='/dli/task/ngc_assets'
os.environ['CLI']='ngccli_linux.zip'

# Remove previous versions of NGC CLI, copy, and install NGC CLI
!rm -r $NGC_DIR/ngccli/*
!cp /dli/task/$CLI $NGC_DIR/ngccli/$CLI
!unzip -u "$NGC_DIR/ngccli/$CLI" \
       -d $NGC_DIR/ngccli/
!rm $NGC_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("NGC_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model download-version nvidia/tao/trafficcamnet:pruned_v1.0 --dest $NGC_DIR

In [ ]:
# Import necessary GStreamer libraries and DeepStream python bindings
import gi
gi.require_version('Gst', '1.0') # gst-launch-1.0
from gi.repository import GObject, Gst, GLib
import pyds

# Initialize GStreamer
Gst.init(None)

# Create Pipeline element that will form a connection of other elements
pipeline=Gst.Pipeline()
print('Created pipeline')

In [ ]:
# Create Source element for reading from a file and set the location property
source=Gst.ElementFactory.make("filesrc", "file-source")
source.set_property('location', target_video_path)

# Create H264 Parser with h264parse as the input file is an elementary h264 stream
h264parser=Gst.ElementFactory.make("h264parse", "h264-parser")

# Create Decoder with nvv4l2decoder for accelerated decoding on GPU
decoder=Gst.ElementFactory.make("nvv4l2decoder", "nvv4l2-decoder")

# Create Streamux with nvstreammux to form batches for one or more sources and set properties
streammux=Gst.ElementFactory.make("nvstreammux", "stream-muxer")
streammux.set_property('width', 888) 
streammux.set_property('height', 696) 
streammux.set_property('batch-size', 1)

# Create Primary GStreamer Inference Element with nvinfer to run inference on the decoder's output after batching
pgie=Gst.ElementFactory.make("nvinfer", "primary-inference")

# Create Sink with fakesink as the end point of the pipeline
fakesink=Gst.ElementFactory.make('fakesink', 'fakesink')
fakesink.set_property('sync', 1)
print('Created elements')

In [ ]:
# Add elements to pipeline
pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(fakesink)
print('Added elements to pipeline')

In [ ]:
!cat spec_files/pgie_config_trafficcamnet_03.txt

In [ ]:
# Set the configuration-file-path property for nvinfer
pgie.set_property('config-file-path', '/dli/task/spec_files/pgie_config_trafficcamnet_03.txt')

In [ ]:
# Link elements in the pipeline
source.link(h264parser)
h264parser.link(decoder)

# Link decoder source pad to streammux sink pad
decoder_srcpad=decoder.get_static_pad("src")    
streammux_sinkpad=streammux.get_request_pad("sink_0")
decoder_srcpad.link(streammux_sinkpad)

# Link the rest of the elements in the pipeline
streammux.link(pgie)
pgie.link(fakesink)
print('Linked elements in pipeline')

In [ ]:
# Declare list to hold count data
obj_counts=[]

# Define the Probe Function
def pgie_src_pad_buffer_probe(pad, info):
    gst_buffer=info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list
    
    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_num=frame_meta.frame_num
        num_obj=frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        print("Frame Number={} Number of Objects={}".format(frame_num, num_obj))
        
        # Append number of objects a list 
        obj_counts.append(num_obj)
        
        # Iterate through each object in the frame metadata until the end
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
                print('\t Object: {} - Top: {}, Left: {}, Width: {}, Height: {}'.format(obj_meta.obj_label, \
                                                                                        round(obj_meta.rect_params.top), \
                                                                                        round(obj_meta.rect_params.left), \
                                                                                        round(obj_meta.rect_params.width), \
                                                                                        round(obj_meta.rect_params.height)))
            except StopIteration:
                break
            
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [ ]:
# Add probe to inference plugin's source
pgie_src_pad=pgie.get_static_pad('src')
probe_id=pgie_src_pad.add_probe(Gst.PadProbeType.BUFFER, pgie_src_pad_buffer_probe)
print('Attached probe')

In [ ]:
from common.bus_call import bus_call

# Inspect the definition for bus_call
from inspect import getsource
print(getsource(bus_call))

In [ ]:
# Create an event loop
loop=GLib.MainLoop()

# Feed GStreamer bus messages to loop
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
# Start play back and listen to events
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

In [ ]:
import pandas as pd

# Export data to a Pandas Series
obj_count_df=pd.Series(obj_counts, name='Object Count')
obj_count_df.index.name='Frame Number'

# Plot the Series
obj_count_df.plot(
    linestyle='none', 
    marker='.', 
    figsize=(15, 5), 
    ylim=[-.1, 3.1], 
    title='Object Count Over Time'
)

In [ ]:
# Import app_3_pt_1.py from sample_apps directory
import sample_apps.app_3_pt_1

# Print the docstring 
print(sample_apps.app_3_pt_1.__doc__)

In [ ]:
from sample_apps.app_3_pt_1 import build_simple_pipeline

# Print the docstring
print(build_simple_pipeline.__doc__)

In [ ]:
# Test run the Python script
!python sample_apps/app_3_pt_1.py data/sample_30.h264

In [ ]:
import time

frame_rates=[]

# Define the Probe Function
def pgie_src_pad_buffer_probe_fps(pad, info):
    global start
    frame_number=0
    gst_buffer=info.get_buffer()
    
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list
    
    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        now=time.time()
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        frame_number=frame_meta.frame_num
        
        # Take the reciprocal of the time difference as frame rate
        frame_rate=round(1/(now-start), 2)
        print('FPS: {} @ Frame {}.'.format(frame_rate, frame_number))
        
        # Add frame rate to frame_rates list
        frame_rates.append(frame_rate)
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
        start=now
    return Gst.PadProbeReturn.OK

In [ ]:
# Standard GStreamer initialization
Gst.init(None)

# Build pipeline
pipeline=build_simple_pipeline('data/sample_30.h264')
print('Successfully created a {} object'.format(type(pipeline)))

# Get the nvinfer plugin by name
pgie=pipeline.get_by_name('primary-inference')

# Add probe to inference plugin's source
pgie_src_pad=pgie.get_static_pad('src')
pgie_src_pad.add_probe(Gst.PadProbeType.BUFFER, pgie_src_pad_buffer_probe_fps)
print('Attached Probe')

In [ ]:
# Create MainLoop and add a signal watch
loop=GLib.MainLoop()
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
# Start Pipeline
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
start=time.time()
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

In [ ]:
# Export data to a Pandas Series
frame_rates_df=pd.Series(frame_rates, name='Frame Rate')
frame_rates_df.index.name='Frame Number'

# Plot the Series
frame_rates_df.plot(
    linestyle='none', 
    marker='.', 
    figsize=(15, 5), 
    ylim=[-.1, 50], 
    title='Frame Rate Over Time')

In [ ]:
!GST_DEBUG='GST_SCHEDULING:7' GST_DEBUG_FILE='/dli/task/logs/trace.log' python sample_apps/app_3_pt_1.py data/sample_30.h264

In [ ]:
!head /dli/task/logs/trace.log

In [ ]:
# Import dependencies
import re

trace_log=[]
# Column headers per documentation
headers=['time_stamp', 'process_id', 'thread_id', 'level', 'category', 'src_file_line', 'function', 'object_name', 'message']

# Helper function to remove ANSI escape sequences
def escape_ansi(line):
    ansi_escape = re.compile(r'(?:\x1B[@-_]|[\x80-\x9F])[0-?]*[ -/]*[@-~]')
    return ansi_escape.sub('', line)

# Open trace.log
with open('/dli/task/logs/trace.log') as f: 
    # Read file
    lines=f.readlines()
    # Iterate through each line
    for each_line in lines: 
        # Exclude the last character, which is a newline (\n) character
        current_line=escape_ansi(each_line[:-1])
        # Split based on white space(s), keeping in mind that src_file, line, function, and object are concatenated together
        time_stamp, process_id, thread_id, level, category, src_file_line_function_object, message=re.split(' +', current_line, maxsplit=6)
        # Split src_file, line, function, and object based on the semicolon character
        src_file, line, function, object=src_file_line_function_object.split(':', maxsplit=3)
        # Add all data to the trace_log list
        trace_log.append([time_stamp, process_id, thread_id, level, category, f'{src_file}:{line}', function, object, message])

# Export data to a DataFrame
df=pd.DataFrame(trace_log, columns=headers)
# Preview the dataframe
df.head()

In [ ]:
# Iterate through rows backwards to get the time stamp
for idx, row in df[::-1].iterrows(): 
    # Time stamp is pts if object is a sink
    if row['object_name'] in ['<stream-muxer:sink_0>', '<primary-inference:sink>', '<fakesink:sink>']: 
        try: 
            df.loc[idx, 'frame_ts']=re.findall('pts \d+:\d+:\d+.\d+', row['message'])[0].split('pts ')[-1]
        except: 
            pass
    # Time stamp is dts if object is a decoder sink
    elif row['object_name']=='<nvv4l2-decoder:sink>': 
        try: 
            ts=re.findall('dts \d+:\d+:\d+.\d+', row['message'])[0].split('dts ')[-1]
            if ts: 
                df.loc[idx, 'frame_ts']=ts
                decoder_offset=re.findall('offset \d+', row['message'])[0].split('offset ')[-1]
        except: 
            pass
    # Time stamp is same as dts of decoder with same offset for file source
    elif row['object_name']=='<file-source:src>':
        try: 
            src_offset=re.findall('offset \d+', row['message'])[0].split('offset ')[-1]
            if src_offset==decoder_offset: 
                df.loc[idx, 'frame_ts']=ts
        except: 
            pass

In [ ]:
time_df=df[['time_stamp', 'object_name', 'frame_ts']].dropna().drop_duplicates(subset=['object_name', 'frame_ts'])

# Pivot dataframe
time_df=time_df.pivot(index='object_name', values='time_stamp', columns='frame_ts')

# Leave time stamp as only seconds 
time_df.columns=[float(each_column.split(':')[2]) for each_column in time_df.columns]

# Clean up
time_df=time_df.dropna(axis=1)

# Display time_df
time_df=time_df.sort_values(0.0).applymap(lambda x: float(x.rsplit(':')[2]))
print('Time Stamp when Buffer Arrives (seconds)')
display(time_df) 

In [ ]:
# Calculate time difference as processed time
diff_df=-time_df.diff(-1).T

# Plot results
diff_df.iloc[:, :-1].plot(figsize=(15, 5)).legend(loc='upper right')

In [ ]:
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 100 > '/dli/task/logs/smi.log' \
&python sample_apps/app_3_pt_1.py data/sample_30.h264

In [ ]:
!cat /dli/task/logs/smi.log

In [ ]:
# Initialize GStreamer
Gst.init(None)

# Build pipeline
pipeline=build_simple_pipeline('data/sample_30.h264')
print('Successfully created a {} object'.format(type(pipeline)))

In [ ]:
# Remove Fakesink
fakesink=pipeline.get_by_name('fakesink')
pipeline.remove(fakesink)

# Create Convertor to convert from YUV to RGBA as required by nvdsosd
nvvidconv1=Gst.ElementFactory.make("nvvideoconvert", "convertor1")

# Create OSD with nvdsosd to draw on the converted RGBA buffer
nvosd=Gst.ElementFactory.make("nvdsosd", "onscreendisplay")

# Create Convertor to convert from RGBA to I420 as required by encoder
nvvidconv2=Gst.ElementFactory.make("nvvideoconvert", "convertor2")

# Create Capsfilter to enforce frame image format
capsfilter=Gst.ElementFactory.make("capsfilter", "capsfilter")
caps=Gst.Caps.from_string("video/x-raw, format=I420")
capsfilter.set_property("caps", caps)

# Create Encoder to encode I420 formatted frames using the MPEG4 codec
encoder = Gst.ElementFactory.make("avenc_mpeg4", "encoder")
encoder.set_property("bitrate", 2000000)

# Create Sink and set the location for the output file
filesink=Gst.ElementFactory.make('filesink', 'filesink')
filesink.set_property('location', 'output_03_encoded.mpeg4')
filesink.set_property("sync", 1)
print('Created elements')

In [ ]:
# Add elements to pipeline
pipeline.add(nvvidconv1)
pipeline.add(nvosd)
pipeline.add(nvvidconv2)
pipeline.add(capsfilter)
pipeline.add(encoder)
pipeline.add(filesink)
print('Added elements to pipeline')

In [ ]:
# Get the nvinfer plugin by name
pgie=pipeline.get_by_name('primary-inference')

# Link elements together
pgie.link(nvvidconv1)
nvvidconv1.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(capsfilter)
capsfilter.link(encoder)
encoder.link(filesink)
print('Linked elements in pipeline')

In [ ]:
from random import random

# Define the Probe Function
def osd_sink_pad_buffer_probe(pad, info):
    gst_buffer=info.get_buffer()

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta=pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame=batch_meta.frame_meta_list

    # Iterate through each frame in the batch metadata until the end
    while l_frame is not None:
        try:
            frame_meta=pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_number=frame_meta.frame_num
        num_rects=frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        # Iterate through each object in the frame metadata until the end
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            
            # Set border color (red, green, blue, alpha) to random values
            obj_meta.rect_params.border_color.set(random(), random(), random(), random())
            
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break

        # Acquire display metadata from pool and set number of labels to 1
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels=1
        
        # Set text_params of the display metadata to local variable
        py_nvosd_text_params=display_meta.text_params[0]

        # Setting display text to be shown on screen
        py_nvosd_text_params.display_text="Frame Number={} Number of Objects={}".format(frame_number, num_rects)

        # Use pyds.get_string() to get display_text as string
        # Reading the display_text field here will return the C address of the
        # allocated string. Use pyds.get_string() to get the string content.
        print(pyds.get_string(py_nvosd_text_params.display_text))
        
        # Set the offsets where the string should appear
        py_nvosd_text_params.x_offset=10
        py_nvosd_text_params.y_offset=10

        # Set font, font-color (red, green, blue, alpha), and font-size
        py_nvosd_text_params.font_params.font_name="Serif"
        py_nvosd_text_params.font_params.font_size=15
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)

        # Set text background color (red, green, blue, alpha)
        py_nvosd_text_params.set_bg_clr=1
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)

        # Add to frame metadata
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)

        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [ ]:
# Add probe to nvdsosd plugin's sink
osdsinkpad=nvosd.get_static_pad("sink")
probe_id=osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe)
print('Attached probe')

In [ ]:
# Create an event loop
loop=GLib.MainLoop()

# Feed GStreamer bus messages to loop
bus=pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
print('Added bus message handler')

In [ ]:
# Start play back and listen to events - this will generate the output_03_raw.mpeg4 file
print("Starting pipeline")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass

# Cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

In [ ]:
# Convert MPEG4 video file to MP4 container file
!ffmpeg -i /dli/task/output_03_encoded.mpeg4 /dli/task/output_03.mp4 -y -loglevel quiet

# View the output video
Video("output_03.mp4", width=720)